In [1]:
import numpy as np
train_Resnet50 = np.load('/dbfs/mnt/dogImages/bottleneck_features/DogResnet50DataTrain.npy')
valid_Resnet50 = np.load('/dbfs/mnt/dogImages/bottleneck_features/DogResnet50DataValid.npy')
test_Resnet50 = np.load('/dbfs/mnt/dogImages/bottleneck_features/DogResnet50DataTest.npy')

train_targets = np.load('/dbfs/mnt/dogImages/bottleneck_features/TrainTargets.npy')
valid_targets = np.load('/dbfs/mnt/dogImages/bottleneck_features/ValidTargets.npy')
test_targets = np.load('/dbfs/mnt/dogImages/bottleneck_features/TestTargets.npy')

In [2]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

my_ResNet50_model = Sequential()
my_ResNet50_model.add(GlobalAveragePooling2D(input_shape=train_Resnet50.shape[1:]))
my_ResNet50_model.add(Dense(134, activation='softmax'))

my_ResNet50_model.summary()

Using TensorFlow backend.
/databricks/python/lib/python3.7/site-packages/botocore/vendored/requests/packages/urllib3/_collections.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
 from collections import Mapping, MutableMapping
WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

Model: "sequential_1"
_________________________________________________________________
Layer (type) Output Shape Param # 
=================================================================
global_average_pooling2d_1 ( (None, 2048) 0 
_________________________________________________________________
dense_1 (Dense) (None, 134) 274566 
=================================================================
Total params: 274,566
Trainable params: 274,566
Non-trainable params: 0
_________________________________________________________________

In [3]:
my_ResNet50_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3576: The name tf.log is deprecated. Please use tf.math.log instead.

In [4]:
from keras.preprocessing.image import ImageDataGenerator

# create and configure augmented image generator
datagen_train = ImageDataGenerator(
    width_shift_range=0.1,  # randomly shift images horizontally (10% of total width)
    height_shift_range=0.1,  # randomly shift images vertically (10% of total height)
    horizontal_flip=True, # randomly flip images horizontally
    shear_range=0.2,
    zoom_range=0.2,) 

# create and configure augmented image generator
datagen_valid = ImageDataGenerator(
    width_shift_range=0.1,  # randomly shift images horizontally (10% of total width)
    height_shift_range=0.1,  # randomly shift images vertically (10% of total height)
    horizontal_flip=True, # randomly flip images horizontally
    shear_range=0.2,
    zoom_range=0.2,) 

# fit augmented image generator on data
datagen_train.fit(train_Resnet50)
datagen_valid.fit(valid_Resnet50)

from keras.callbacks import ModelCheckpoint

batch_size = 20
epochs = 20

checkpointer = ModelCheckpoint(filepath='weights.best.CifarResNet-50.hdf5', 
                               verbose=1, save_best_only=True)

my_ResNet50_model.fit_generator(datagen_train.flow(train_Resnet50, train_targets, batch_size=batch_size),
                    steps_per_epoch=train_Resnet50.shape[0] // batch_size,
                    epochs=epochs, callbacks=[checkpointer],
                    validation_data=datagen_valid.flow(valid_Resnet50, valid_targets, batch_size=batch_size),
                    validation_steps= valid_Resnet50.shape[0] // batch_size,
                    verbose=1)

In [5]:
from keras.callbacks import ModelCheckpoint

batch_size = 20
epochs = 20

checkpointer = ModelCheckpoint(filepath='weights.best.DogResNet-50.hdf5', 
                               verbose=1, save_best_only=True)

my_ResNet50_model.fit(train_Resnet50, train_targets, 
          validation_data=(valid_Resnet50, valid_targets),
          epochs=20, batch_size=20, callbacks=[checkpointer], verbose=1)

In [6]:
my_ResNet50_model.load_weights('weights.best.DogResNet-50.hdf5')

In [7]:
my_ResNet50_predictions = [np.argmax(my_ResNet50_model.predict(np.expand_dims(feature, axis=0))) for feature in test_Resnet50]

# report test accuracy
test_accuracy = 100*np.sum(np.array(my_ResNet50_predictions)==np.argmax(test_targets, axis=1))/len(my_ResNet50_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 81.4947%

In [8]:
import mlflow.keras
# save the mode to local or remote accessible path on the S3 or Azure Blob
mlflow.keras.save_model(my_ResNet50_model, '/dbfs/mnt/dogImages/saved_model/dog_resnet50')

In [9]:
import mlflow.keras
from keras.applications.resnet50 import ResNet50
# save the mode to local or remote accessible path on the S3 or Azure Blob
mlflow.keras.save_model(ResNet50(weights='imagenet', include_top=False), '/dbfs/mnt/dogImages/saved_model/notop_resnet50')

Using TensorFlow backend.
/databricks/python/lib/python3.7/site-packages/botocore/vendored/requests/packages/urllib3/_collections.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
 from collections import Mapping, MutableMapping
WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4479: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:197: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:203: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:207: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:216: The name tf.is_variable_initialized is deprecated. Please use tf.compat.v1.is_variable_initialized instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:223: The name tf.variables_initializer is deprecated. Please use tf.compat.v1.variables_initializer instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:2041: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:148: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4267: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

/databricks/python/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
 warnings.warn('The output shape of `ResNet50(include_top=False)` '
Downloading data from https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5

 8192/94653016 [..............................] - ETA: 17s
 1622016/94653016 [..............................] - ETA: 3s 
 7176192/94653016 [=>............................] - ETA: 1s
18382848/94653016 [====>.........................] - ETA: 0s
27099136/94653016 [=======>......................] - ETA: 0s
32161792/94653016 [=========>....................] - ETA: 0s
37224448/94653016 [==========>...................] - ETA: 0s
42205184/94653016 [============>.................] - ETA: 0s
47308800/94653016 [=============>................] - ETA: 0s
52322304/94653016 [===============>..............] - ETA: 0s
57319424/94653016 [=================>............] - ETA: 0s
62349312/94653016 [==================>...........] - ETA: 0s
64831488/94653016 [===================>..........] - ETA: